In [11]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import pygsheets

from ipynb.fs.full.xscores_selenium_functions import scrape_xscores_completed
from ipynb.fs.full.xscores_selenium_functions import scrape_xscores_upcoming

from YourInfo import folder_loc

In [1]:
# for reference in cron log
from datetime import date
print('Date of script run: ', date.today(), '\n')

Date of script run:  2023-03-09


---
### This script is designed to scrape xscores.com for results from the current soccer season in Europe's top leagues, and compile with results from past seasons, then export the data as a .csv file.
---

In [12]:
#Leagues and countries of interest:

league_names = pd.DataFrame({
    'country':['spain', 'england','germany','france','italy'],
    'league_name':['primera-division','premier-league','bundesliga','ligue-1','serie-a']
})
cup_names = pd.DataFrame({
    'country':['europe-uefa','europe-uefa','spain','england','germany','france', 'italy'],
    'league_name':['uefa-champions-league','uefa-europa-league','fa-cup','fa-cup','fa-cup','fa-cup','fa-cup']
})
leagues_cups = pd.concat([league_names,cup_names])
leagues_cups


,country,league_name
0,spain,primera-division
1,england,premier-league
2,germany,bundesliga
3,france,ligue-1
4,italy,serie-a
0,europe-uefa,uefa-champions-league
1,europe-uefa,uefa-europa-league
2,spain,fa-cup
3,england,fa-cup
4,germany,fa-cup


In [13]:
#scrape completed and upcoming games for each league, compile into current season data

year = 2022   #select year of ongoing season (currently the 2022-2023 season)

data = pd.DataFrame(columns=['Round', 'Date', 'Time', 'Home_Team', 'Home_Score', 'Away_Score', 'Away_Team',
                                'Home_Score_AET', 'Away_Score_AET', 'Home_Penalties', 'Away_Penalties',
                                'Home_Points', 'Away_Points','season', 'Country', 'Competition'])

for i in range(len(leagues_cups)):
    print('start: '+ leagues_cups.iloc[i,0] +' '+ leagues_cups.iloc[i,1])
    completed = scrape_xscores_completed(year, leagues_cups.iloc[i,0], leagues_cups.iloc[i,1])
    print('len(completed): ',len(completed))
    upcoming = scrape_xscores_upcoming(leagues_cups.iloc[i,0], leagues_cups.iloc[i,1])
    print('len(upcoming): ',len(upcoming))
    
    current_season = pd.concat( [completed, upcoming] )
    current_season['season'] = year
    current_season['Country'] = leagues_cups.iloc[i,0]       #add country and competition indicator columns
    current_season['Competition'] = leagues_cups.iloc[i,1]
    
    data = pd.concat([data, current_season])
    
    print('end: '+ leagues_cups.iloc[i,0] +' '+ leagues_cups.iloc[i,1]+'\n')

start: spain primera-division

len(completed):  240

len(upcoming):  140
end: spain primera-division
start: england premier-league

len(completed):  255

len(upcoming):  125
end: england premier-league
start: germany bundesliga

len(completed):  207

len(upcoming):  99
end: germany bundesliga
start: france ligue-1

len(completed):  260

len(upcoming):  120
end: france ligue-1
start: italy serie-a

len(completed):  250

len(upcoming):  130
end: italy serie-a
start: europe-uefa uefa-champions-league

len(completed):  12

len(upcoming):  4
end: europe-uefa uefa-champions-league
start: europe-uefa uefa-europa-league

len(completed):  24

len(upcoming):  8
end: europe-uefa uefa-europa-league
start: spain fa-cup

len(completed):  113

len(upcoming):  3
end: spain fa-cup
start: england fa-cup

len(completed):  137

len(upcoming):  4
end: england fa-cup
start: germany fa-cup

len(completed):  56

len(upcoming):  4
end: germany fa-cup
start: france fa-cup

len(completed):  334

len(upcoming):  

In [14]:
#scrape group stage of european competitions seperately (due to complications with URLs):

for i in range(5,7):
    league_url_form = leagues_cups.iloc[i,1] + '/group-stage'
    current_season = scrape_xscores_completed(year, leagues_cups.iloc[i,0], league_url_form)
    
    current_season['season'] = year
    current_season['Country'] = leagues_cups.iloc[i,0]       #add country and competition indicator columns
    current_season['Competition'] = leagues_cups.iloc[i,1]
    
    data = pd.concat([data, current_season])

In [15]:
#Compile current season data with data from past 20 years, and transform to work with in Tableau.

past_seasons = pd.read_csv( folder_loc+'Data/Past_20years_Data.csv' )

raw_data = pd.concat([past_seasons, data])
    
home = raw_data.rename(columns = {
        'Home_Team':'Team',
        'Away_Team':'Opponent',
        'Home_Score':'Team_Score',
        'Away_Score':'Opponent_Score',
        'Home_Points':'Team_Points',
        'Away_Points':'Opponent_Points'
})
home['Location'] = 'Home'
    
away = raw_data.rename(columns = {
        'Away_Team':'Team',
        'Home_Team':'Opponent',
        'Away_Score':'Team_Score',
        'Home_Score':'Opponent_Score',
        'Away_Points':'Team_Points',
        'Home_Points':'Opponent_Points'
})
away['Location'] = 'Away'

full_dataset = pd.concat([home, away], ignore_index=True)
    
full_dataset.to_csv( folder_loc+'Data/Full_Dataset.csv',index=False)

In [16]:
#export updated data to google sheets via pygsheets

# google sheets authentication
# this json file had all the info to get access to a gmail account which is shared on the google sheets document
creds = folder_loc+'pygsheets_api_authorization_file.json'
api = pygsheets.authorize(service_file=creds)

# open the workbook 'Soccer Data' from google sheets, and access 'sheet1'
sheet = api.open('Soccer Data').sheet1

#read data
data = pd.read_csv( folder_loc+'Data/Full_Dataset.csv' )

#assign data to google sheets
sheet.set_dataframe(data, (1,1), fit=True)